In [24]:
import numpy as np
import torch

# einsumと仲良くなろうの会

## einsumとは
アインシュタインの縮約記号。高階テンソル同士の演算が多用される物理系ではよく使われる記法で、大量の$\Sigma$を省略できる。

PyTorchでは

`torch.einsum('ik, kj -> ij', A, B)`のように使える。
`'ik, kj -> ij'`部分を(ドキュメントに従って) `equation`と呼ぶことにする。

# ルール
- equationは
`i -> k`
`i, j -> k`
`ik, kj -> ij`
のように、`() -> ()`の形で、アルファベットでインデックスを指定するように書く。

- 各アルファベットに意味があるわけではなく、例えば
`i -> k`
と
`j -> l`
は等価で、また大文字でも良い。
(e.g. `I -> J`)

- コンマで区切って各引数に対して演算を定義する。
各引数に関して、次元とアルファベットの数は一致する。
(例えば、行列に対しては`ij`や`jk`など)

- equation両辺の次元が合うように $\Sigma$が適当に補われた式として解釈される

- 右辺が省略されているときはスカラーと解釈される

#### 1次元配列(ベクトル) -> スカラの場合

In [25]:
a = torch.tensor([1, 2, 3])

In [26]:
torch.einsum('i->', a)

tensor(6)

In [27]:
# これと等価
s = 0
for i in range(3):
    s += a[i]
    
print(s)

tensor(6)


#### 二次元配列(行列) -> スカラ の場合

In [28]:
a = torch.ones(3, 3)
a

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [29]:
torch.einsum('ij->', a)

tensor(9.)

In [30]:
# これと等価
s = 0
for i in range(3):
    for j in range(3):
        s += a[i, j]
    
print(s)

tensor(9.)


#### 3次元配列(3階テンソル) -> スカラの場合

In [31]:
a = torch.ones(3, 3, 3)

In [32]:
torch.einsum('ijk ->', a)

tensor(27.)

In [33]:
s = 0
for i in range(3):
    for j in range(3):
        for k in range(3):
            s += a[i, j, k]
    
print(s)

tensor(27.)


右辺がスカラではない場合(ベクトルや行列)、とりあえず`ij -> k`のようなものが思いつくのでとりあえず書いてみる

In [34]:
a = torch.tensor(np.arange(0, 9).reshape(3, 3))
a

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [35]:
b = torch.einsum('ij->k', a)
b

RuntimeError: einsum(): output subscript k does not appear in the equation for any input operand

よくよく考えてみれば、`ij -> k`という添字を持つような式を $\Sigma$を加えることで構成することは難しそう。となると、こうなる

In [36]:
b = torch.einsum('ij->i', a)

このequationは、

$$
\sum_j a_{i_j} = b_i
$$

と解釈される

In [37]:
b

tensor([ 3, 12, 21])

#### 問題
1. $tr A$ をeinsumで実装しよう
2. $b_i = A_i{_i}$なる$b_i$を実装しよう
3. 転置を実装しよう

In [38]:
a = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [39]:
trA = torch.einsum('ii->', a)
trA

tensor(15)

In [40]:
b = torch.einsum('ii->i', a)
b

tensor([1, 5, 9])

In [41]:
Aᵗ = torch.einsum('ij->ji', a)
Aᵗ

tensor([[1, 4, 7],
        [2, 5, 8],
        [3, 6, 9]])